In [1]:
import json 
all_tables = []
with open("/home/suyash/final_repo/fetaqa_MM_cleaned/new_outputs/og_tables/fetaQA-v1_dev.jsonl") as f:
    for line in f:
        all_tables.append(json.loads(line))

with open("/home/suyash/final_repo/fetaqa_MM_cleaned/new_outputs/og_tables/fetaQA-v1_train.jsonl") as f:
    for line in f:
        all_tables.append(json.loads(line))

with open("/home/suyash/final_repo/fetaqa_MM_cleaned/new_outputs/og_tables/fetaQA-v1_test.jsonl") as f:
    for line in f:
        all_tables.append(json.loads(line))

In [2]:
print(len(all_tables))

10330


In [13]:
# write all tables to a file 
with open("/home/suyash/final_repo/fetaqa_MM_cleaned/new_outputs/all_tables.jsonl", "w") as f:
    for table in all_tables:
        f.write(json.dumps(table) + "\n")

In [ ]:
revision_ids_queries = {}
URL = "https://en.wikipedia.org/w/api.php?action=query&titles={title}&prop=revisions&rvlimit=500&rvstart=2020-05-05T07%3A59%3A00Z&rvdir=older&format=json&redirects=1"
for table in all_tables:
    page_title = table['page_wikipedia_url'].split("en.wikipedia.org/wiki/")[1]
    revision_ids_queries[page_title] = URL.format(title=page_title)

In [ ]:
len(revision_ids_queries)

In [ ]:
json.dump(revision_ids_queries, open("/home2/jainit/FeTAQA_MM/new_outputs/input_asyncio/revision_ids_queries.json", "w"))

In [2]:
# now we need to get the table ids for each url and the table array from 
from collections import Counter

def compute_similarity(n_grams_1, n_grams_2):
    counter1 = Counter(n_grams_1)
    counter2 = Counter(n_grams_2)
    
    # Compute intersection and union for multisets
    intersection = sum((counter1 & counter2).values())
    union = sum((counter1 | counter2).values())
    
    # Compute similarity
    similarity = intersection / union if union != 0 else 0
    return similarity

import nltk
import json
import pandas as pd
from bs4 import BeautifulSoup
from nltk.util import ngrams

from nltk import word_tokenize
from tqdm import tqdm
file_path = "/home/suyash/final_repo/fetaqa_MM_cleaned/output/all_tables.jsonl"
rev_cnt=0
most_similar_tables = {}
URL = "https://en.wikipedia.org/w/api.php?action=query&titles={title}&prop=revisions&rvlimit=500&rvstart=2020-05-05T07%3A59%3A00Z&rvdir=older&format=json&redirects=1"

iii=0
different_count = 0
# Open the JSONL file and read line by line
redirect_titles = []
url_to_table_id = json.load(open("/home/suyash/final_repo/fetaqa_MM_cleaned/output/url_to_table_id.json",'r'))

with open(file_path, "r") as jsonl_file:
    for line  in tqdm(jsonl_file) :
       
        json_data = json.loads(line)
        
        # Now you can work with the JSON data as a Python dictionary
        # print(json_data)  # You can replace this with your processing logic
        page_url = json_data['page_wikipedia_url']
        feta_id = json_data['feta_id']
        table_section_title = json_data['table_section_title']
        page_title = json_data['table_page_title']
        raw_title = page_url.split("en.wikipedia.org/wiki/")[1]
        page_url = URL.format(title=raw_title)
   
        table_1_df  = pd.DataFrame(json_data['table_array'])
        words = set()
        for i in range(table_1_df.shape[0]):
            for j in range(table_1_df.shape[1]):
                cell_value = str(table_1_df.iloc[i,j]).lower()
                if len(cell_value.split())>1:
                    n=2 
                    ngramss = set(ngrams(word_tokenize(cell_value), n))
                    words.update(set(ngramss))
                else:
                    words.add(cell_value)
        
        table_id_with_max_similarity = None
        max_similarity = 0
        
        table_id_with_max_similarity_new_strat = None
        max_similarity_new_strat = 0 # Trying out if different results with a newer stratagey
        
        if page_url in url_to_table_id:
            for table_ids in url_to_table_id[page_url]:
                try: 
                    table_2_df = pd.read_html(f"/home/suyash/final_repo/fetaqa_MM_cleaned/output/tables/{table_ids}.html")[0]
                    words2 = set()

                    for i in range(table_2_df.shape[0]):
                        for j in range(table_2_df.shape[1]):
                            cell_value = str(table_2_df.iloc[i,j]).lower()

                            if len(cell_value.split())>1:
                                n=2
                                ngras = set(ngrams(word_tokenize(cell_value), n))
                                words2.update(set(ngras))

                            else :
                                words2.add(cell_value)

                    similarity = len(words.intersection(words2))/len(words.union(words2))
                    if similarity>max_similarity:
                        max_similarity = similarity
                        table_id_with_max_similarity = table_ids
                    
                    different_strat_similarity = compute_similarity(words, words2)
                    if different_strat_similarity>max_similarity_new_strat:
                        max_similarity_new_strat = different_strat_similarity
                        table_id_with_max_similarity_new_strat = table_ids
                    
                    if table_id_with_max_similarity != table_id_with_max_similarity_new_strat:
                        different_count += 1
                        print("AAAAAAAAAAAAAAAAAAAAAAAA different")
                
                except Exception as e:
                    print(e)
                    print( "Path : ", f"/home/suyash/final_repo/fetaqa_MM_cleaned/output/tables/{table_ids}.html")
                    iii+=1 
                    continue
        else : 
            print("not found")
            print(page_url)
        if table_id_with_max_similarity is None:
            print("not found")
            print(page_url)
        most_similar_tables[feta_id] =(page_url,table_id_with_max_similarity, max_similarity) 

print(different_count)


0it [00:00, ?it/s]

10it [00:00, 20.95it/s]

not found
https://en.wikipedia.org/w/api.php?action=query&titles=Natalie_Cohen&prop=revisions&rvlimit=500&rvstart=2020-05-05T07%3A59%3A00Z&rvdir=older&format=json&redirects=1


41it [00:02, 21.89it/s]

not found
https://en.wikipedia.org/w/api.php?action=query&titles=Chapra,_Nadia_(Vidhan_Sabha_constituency)&prop=revisions&rvlimit=500&rvstart=2020-05-05T07%3A59%3A00Z&rvdir=older&format=json&redirects=1


...

not found
https://en.wikipedia.org/w/api.php?action=query&titles=The_D'Ambrosio_Twins&prop=revisions&rvlimit=500&rvstart=2020-05-05T07%3A59%3A00Z&rvdir=older&format=json&redirects=1


10094it [34:26,  3.92it/s]

No tables found matching pattern '.+'
Path :  /home/suyash/final_repo/fetaqa_MM_cleaned/output/tables/57132.html
not found
https://en.wikipedia.org/w/api.php?action=query&titles=The_Holographic_Principle_World_Tour&prop=revisions&rvlimit=500&rvstart=2020-05-05T07%3A59%3A00Z&rvdir=older&format=json&redirects=1


10330it [35:03,  4.91it/s]

0


In [ ]:
# Using Jaccard's coeffiecient without converting to set led to same results
json.dump(most_similar_tables, open("/home2/jainit/FeTAQA_MM/new_outputs/most_similar_tables.json", "w")) 

### Next step

In [22]:
# now we need to get the table ids for each url and the table array from 
from tqdm import tqdm
from bs4 import BeautifulSoup
import json
most_similar_tables= json.load(open("/home/suyash/final_repo/fetaqa_MM_cleaned/output/most_similar_tables.json", "r")) 

table_links = {}
for feta_id, table_info in tqdm(most_similar_tables.items()):
    page_url = table_info[0]
    table_id = table_info[1]
    if table_id is None:
        continue

    with open(f"/home/suyash/final_repo/fetaqa_MM_cleaned/output/tables/{table_id}.html", "r") as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')
        tables = soup.find_all('table')

        for table in tables:
            rows = table.find_all('tr')
            for row in rows:
                # print("-----------------------")
                # print(row)
                # print("-----------------------")
                cells = row.find_all('td')
                # print(cells)
                for cell in cells:
                    links = cell.find_all('a')

                    for link in links:
                        if link.get('href') is None:
                            continue

                        link_url = link.get('href')
                        # print("=======================")
                        # print(cell)
                        # print(link_url)
                        # print("========================")
                        if link_url.startswith("/wiki") or (link_url.startswith("https") and "/wiki/" in link_url):
                            table_links.setdefault(table_id, {})
                            # TODO Suyash: Remove the lower here and rerun!!!!
                            temp_text = link.text.lower().strip().replace("\n", " ").replace("\t", " ").replace("\r", " ").replace("  ", " ")
                            
                            if temp_text not in table_links[table_id]:
                                table_links[table_id][temp_text] = set()
                            table_links[table_id][temp_text].add(link_url)


100%|██████████| 10330/10330 [03:46<00:00, 45.66it/s]


In [25]:
troubled_links = 0
total_links = 0
for table_id, text_dict in table_links.items():
    text_array = sorted(text_dict.items(), key=lambda item: len(item[0]) , reverse=True)
    for string, links in text_array:
        total_links += len(links)
        if len(links) > 1 and string.strip() != "":
            troubled_links += len(links)
            # print(string)
            # print(links)
print(troubled_links)
print(total_links)

14180
166286


In [6]:
import json
from bs4 import BeautifulSoup
most_similar_tables= json.load( open("/home/suyash/final_repo/fetaqa_MM_cleaned/output/most_similar_tables.json", "r")) 

In [7]:
new_tables_links = {} # Contains the text -> links sorted by the length of the text. This way the longest strings get replaced first.
for k , v in table_links.items():
    sorted_dict = dict(sorted(v.items(), key=lambda item: len(item[0]) , reverse=True))
    new_tables_links[k] = sorted_dict
    

In [42]:
with open("/home/suyash/final_repo/fetaqa_MM_cleaned/output/table_links.json", "w") as f:
    json.dump(new_tables_links, f)

In [4]:
with open("/home/suyash/final_repo/fetaqa_MM_cleaned/output/table_links.json", "r") as f:
    table_links = json.load(f)

with open("/home/suyash/final_repo/fetaqa_MM_cleaned/output/most_similar_tables.json", "r") as f:
    most_similar_tables = json.load(f)

In [5]:
def calculate_alpha_ratio(input_string):
    
    # remove all spaces
    input_string = input_string.replace(" ", "")
    alpha_count = sum(c.isalpha() for c in input_string)
    non_alpha_count = len(input_string) - alpha_count
    
    if non_alpha_count == 0:
        return 1
    ratio = alpha_count / non_alpha_count
    # print("Ratio", ratio)
    return ratio


In [6]:
import json

with open("/home/suyash/final_repo/fetaqa_MM_cleaned/output/download_paths.json", "r") as f:
    table_links_to_downloaded_paths = json.load(f)

with open('/home/suyash/final_repo/fetaqa_MM_cleaned/output/link_to_full_links.json', 'r') as f:
    links_to_full_links = json.load(f)

with open("/home/suyash/final_repo/fetaqa_MM_cleaned/output/link_to_single_image.json", "r") as f:
    link_to_single_image = json.load(f)

tables = []

In [7]:
# now we create a new dict which will have the link to the image_path
link_to_image_path = {}
for k , v in links_to_full_links.items():
    if v in link_to_single_image:
        if link_to_single_image[v] in table_links_to_downloaded_paths:
            link_to_image_path[k] = table_links_to_downloaded_paths[link_to_single_image[v]]

In [8]:
import re
def replace_with_condition(cell_string, replace_str, replacement_str):
    # Define the regular expression pattern to match replace_str with non-alphanumeric or no adjacent characters
    cell_string = " " + cell_string + " "
    pattern = r'([^a-zA-Z0-9])' + re.escape(replace_str) + r'([^a-zA-Z0-9])'
    
    # Use re.sub to replace all matches in the cell_string with replacement_str with case-insensitive matching
    result = re.sub(pattern, r'\1' + replacement_str + r'\2', cell_string, flags=re.IGNORECASE)
    is_unchanged = cell_string==result
    return (result.strip(), is_unchanged)

In [9]:
replace_with_condition("w a ksjdkasjdfk", "kjs", "ksjdkasjdfk")

('w a ksjdkasjdfk', True)

In [10]:
import re 
def calculate_alpha_ratio(input_string):
    
    # remove all spaces
    input_string = input_string.replace(" ", "")
    alpha_count = sum(c.isalpha() for c in input_string)
    non_alpha_count = len(input_string) - alpha_count
    
    if non_alpha_count == 0:
        return 1
    ratio = alpha_count / non_alpha_count
    # print("Ratio", ratio)
    return ratio

In [11]:
# now we need to replace the entities in teh table with {{entity}{link_to_wiki_page}}
import json 
from tqdm import tqdm
from bs4 import BeautifulSoup
import re
import copy

with open("/home/suyash/final_repo/fetaqa_MM_cleaned/output/most_similar_tables.json", "r") as f:
    table_similarites = json.load(f)

with open("/home/suyash/final_repo/fetaqa_MM_cleaned/output/table_links.json", "r") as f:
    table_links = json.load(f)

file_path = "/home/suyash/final_repo/fetaqa_MM_cleaned/output/all_tables.jsonl"

questions_f  = open("/home/suyash/final_repo/fetaqa_MM_cleaned/final_data_unfiltered/questions.jsonl", "w")
tables_f = open("/home/suyash/final_repo/fetaqa_MM_cleaned/final_data_unfiltered/tables.jsonl", "w")

i=0
j=0 
missing_cnt = 0

image_ids_to_string = {}
image_ids_to_download_path = {}
image_ids_to_links_page = {}

NOTES_ARRAY = ["note", "comment"]

with open(file_path, "r") as jsonl_file:
    for line  in (jsonl_file):
        json_data = json.loads(line)
        page_url = json_data['page_wikipedia_url']
        table = copy.deepcopy(json_data['table_array'])
        feta_id = json_data['feta_id']
        
        # now we get the table_id for this fetad_id and get the table and then get the links for each cell
        if str(json_data['feta_id']) not in table_similarites:
            print("WTF!", json_data['feta_id'])
            continue
        
        table_id = table_similarites[str(json_data['feta_id'])][1]
        new_tables = table.copy()
        if table_id is None:
            continue
        
        if str(table_id) not in table_links:
            missing_cnt+=1
            continue
        
        table_headers = table[0]
        
        for row_id , row in enumerate(table):
            for col_id, cell in enumerate(row):
                image_id = 0 
                cell = cell.strip().replace("\n", " ").replace("\t", " ").replace("\r", " ").replace("  ", " ")
                
                if cell!= "":
                    for text,link  in table_links[str(table_id)].items():
                        text = text.strip().replace("\n", " ").replace("\t", " ").replace("\r", " ").replace("  ", " ")
                        alpha_ratio =  calculate_alpha_ratio(text)
                        
                        if alpha_ratio<0.5 and len(text)<4:
                            # if link in image_ids_to_download_path:
                                # print("---------------------------")
                                # print(text)
                                # print(link)
                                # print(feta_id)
                                # print("---------------------------")
                            continue
                        
                        if text != "" and len(text)>3 and link in link_to_image_path and "/wiki/" in link and any([elem in table_headers[col_id].lower() for elem in NOTES_ARRAY]) == 0: #and  bool(re.search(r'[a-zA-Z]', text)):
                            image_str = f"{{IMG-{{FTQ-{feta_id}-{row_id}-{col_id}-{image_id}}}}}"
                            
                            cell, is_unchanged = replace_with_condition(cell, text, image_str)
                            if not is_unchanged:
                                if alpha_ratio<0.5 and len(text)>4:
                                    pass
                                
                                
                                image_ids_to_string[image_str] = text
                                image_ids_to_download_path[image_str] = link_to_image_path[link]
                                image_ids_to_links_page[image_str] = link
                                
                                image_id += 1
                                i+=1 
                            # break
                new_tables[row_id][col_id] = cell
            
                # print("----------------------------------")
                        
        json_data['table_array_with_image_id'] = new_tables
        table_id = f"FTQ-{feta_id}"
        tables_dict = {
            "table_id": table_id,
            "table_array": new_tables,
            "original_table_array": table,
            "table_source_json": json_data["table_source_json"],
            "page_wikipedia_url": json_data["page_wikipedia_url"],
            "table_page_title": json_data["table_page_title"],
            "table_section_title": json_data["table_section_title"],
        }
        questions_dict = {
            "question_id": feta_id,
            "table_context": table_id,
            "question": json_data['question'],
            "answer": json_data['answer'],
            "highlighted_cell_ids": json_data['highlighted_cell_ids'],
        }
        tables_f.write(json.dumps(tables_dict)+"\n")
        questions_f.write(json.dumps(questions_dict)+"\n")
        # print("wriiten")
tables_f.close()
questions_f.close()

with open("/home/suyash/final_repo/fetaqa_MM_cleaned/final_data_unfiltered/image_ids_to_string.json", "w") as f:
    json.dump(image_ids_to_string, f)

with open("/home/suyash/final_repo/fetaqa_MM_cleaned/final_data_unfiltered/image_ids_to_download_path.json", "w") as f:
    json.dump(image_ids_to_download_path, f)

with open("/home/suyash/final_repo/fetaqa_MM_cleaned/final_data_unfiltered/image_ids_to_links_page.json", "w") as f:
    json.dump(image_ids_to_links_page, f)

print(missing_cnt)

1003
